### Notebook configuration

In [3]:
run_numbers         = 4735, 4737, 4739
correction_filename =  f"$IC_DATA/XYmaps/corrections_run4734.h5"
input_dst_filenames = [f"$IC_DATA/Cs137/kdst_{run_number}.h5" for run_number in run_numbers]
output_dst_filename =   "$IC_DATA/Kr/dst_{}.root.h5".format("_".join(map(str, run_numbers)))

### Imports

In [14]:
import os
import glob
import functools

import numpy  as np
import tables as tb

import invisible_cities.reco.dst_functions as dstf
import invisible_cities.io  .dst_io        as dstio

### Initialization

In [17]:
input_dst_filenames = list(map(os.path.expandvars, input_dst_filenames))
correction_filename =          os.path.expandvars(correction_filename) 
output_dst_filename =          os.path.expandvars(output_dst_filename)

### Load dsts

In [10]:
dsts = list(map(functools.partial(dstio.load_dst, group="DST", node="Events"), input_dst_filenames))

In [ ]:
ELT_correction  = dstf.load_lifetime_xy_corrections(correction_filename,
                                                    group = "XYcorrections",
                                                    node  = "Lifetime")


In [25]:
def compute_average_lifetime(xlim, ylim, N=1000000):
    lts = dstf.load_xy_corrections(correction_filename,
                                   group = "XYcorrections",
                                   node  = "Lifetime")
    x = np.random.uniform(*xlim, size=N)
    y = np.random.uniform(*ylim, size=N)
    l = lts(x, y).value
    return np.mean(l), np.std(l, ddof=1)

In [33]:
xlim   = -10, 10
ylim   = -10, 10
lt_ref, lt_ref_std = compute_average_lifetime(xlim, ylim)
lt_ref

1801.7300964484864

In [ ]:
def 

### Create new dst

In [10]:
# Unfortunately, this method can't set a specific name to the table or its title.
# It also includes an extra column ("index") which I can't manage to remove.
subdst = dst[one_S1]
subdst.to_hdf(output_dst_filename,
              key     = "DST"  , mode         = "w",
              format  = "table", data_columns = True,
              complib = "zlib" , complevel    = 4)

# Workaround to re-establish the name of the table and its title
with tb.open_file(output_dst_filename, "r+") as f:
    f.rename_node(f.root.DST.table, "Events")
    f.root.DST.Events.title = "Events"